### Packages Required

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import MinMaxScaler 

In [2]:
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout
import tensorflow as tf

### Datasets

In [3]:
path = 'C:/Users/Nithin/Downloads/Google Stock Prediction_AI_Forecasting/'

In [4]:
train_df = pd.read_csv(path+'Datasets/train.csv')
test_df = pd.read_csv(path+'Datasets/test.csv')

In [6]:
train = train_df.iloc[:,1:2].values

In [8]:
test = test_df.iloc[:,1:2].values
total_df = pd.concat([train_df['Open'],test_df['Open']],axis=0)

In [9]:
total_df[len(total_df)-len(test)-60:].values

array([ 955.48999 ,  966.700012,  980.      ,  980.      ,  973.719971,
        987.450012,  992.      ,  992.099976,  990.289978,  991.77002 ,
        986.      ,  989.440002,  989.52002 ,  970.      ,  968.369995,
        980.      , 1009.190002, 1014.      , 1015.219971, 1017.210022,
       1021.76001 , 1022.109985, 1028.98999 , 1027.27002 , 1030.52002 ,
       1033.98999 , 1026.459961, 1023.419983, 1022.590027, 1019.210022,
       1022.52002 , 1034.01001 , 1020.26001 , 1023.309998, 1035.      ,
       1035.869995, 1040.      , 1055.089966, 1042.680054, 1022.369995,
       1015.799988, 1012.659973,  995.940002, 1001.5     , 1020.429993,
       1037.48999 , 1035.5     , 1039.630005, 1046.119995, 1045.      ,
       1054.609985, 1066.079956, 1075.199951, 1071.780029, 1064.949951,
       1061.109985, 1058.069946, 1057.390015, 1051.599976, 1046.719971,
       1048.339966, 1064.310059, 1088.      , 1094.      , 1102.22998 ,
       1109.400024, 1097.099976, 1106.300049, 1102.410034, 1132.

In [10]:
inputs = total_df[len(total_df)-len(test)-60:].values

**-- Standardise the data**

In [12]:
sc = MinMaxScaler(feature_range=(0,1))
train_std = sc.fit_transform(train)

In [23]:
inputs = inputs.reshape(-1,1)
test_std = sc.transform(inputs)

In [13]:
train_x = []
train_y = []

for i in range(60,train_std.size):
    train_x.append(train_std[i-60:i,0])
    train_y.append(train_std[i,0])
    
train_x,train_y = np.array(train_x),np.array(train_y)

In [24]:
test_x = []

for i in range(60,test_std.size):
    test_x.append(inputs[i-60:i,0])
    
test_x = np.array(test_x)

In [18]:
train_x = np.resize(train_x,[train_x.shape[0],train_x.shape[1],1])

In [25]:
test_x = np.resize(test_x,[test_x.shape[0],test_x.shape[1],1])

### Model Building

In [20]:
#--> Intialise Model
M0 = Sequential()
#--> 1st Layer
M0.add(LSTM(units=50,return_sequences=True,input_shape=(train_x.shape[1],1)))
M0.add(Dropout(0.2))
#--> 2nd Layer
M0.add(LSTM(units=50,return_sequences=True))
M0.add(Dropout(0.2))
#--> 3rd Layer
M0.add(LSTM(units=50,return_sequences=True))
M0.add(Dropout(0.2))
#--> 4th Layer
M0.add(LSTM(units=50,return_sequences=True))
M0.add(Dropout(0.2))
#--> 5th Layer
M0.add(LSTM(units=50))
M0.add(Dropout(0.2))
#-->Output layer
M0.add(Dense(units=1))

#--> Compile
M0.compile(optimizer='adam',loss='mean_squared_error')

In [21]:
M0.fit(train_x,train_y,epochs=100,batch_size=32)

Epoch 1/100
38/38 [==============================] - 9s 72ms/step - loss: 0.0270
Epoch 2/100
38/38 [==============================] - 3s 71ms/step - loss: 0.0048
Epoch 3/100
38/38 [==============================] - 3s 67ms/step - loss: 0.0047
Epoch 4/100
38/38 [==============================] - 3s 70ms/step - loss: 0.0042
Epoch 5/100
38/38 [==============================] - 3s 72ms/step - loss: 0.0039
Epoch 6/100
38/38 [==============================] - 3s 88ms/step - loss: 0.0039
Epoch 7/100
38/38 [==============================] - 3s 85ms/step - loss: 0.0040
Epoch 8/100
38/38 [==============================] - 3s 91ms/step - loss: 0.0035
Epoch 9/100
38/38 [==============================] - 3s 87ms/step - loss: 0.0034
Epoch 10/100
38/38 [==============================] - 3s 85ms/step - loss: 0.0041
Epoch 11/100
38/38 [==============================] - 3s 87ms/step - loss: 0.0034
Epoch 12/100
38/38 [==============================] - 3s 89ms/step - loss: 0.0032
Epoch 13/100
38/38 [=====

In [27]:
pred = M0.predict(test_x)
test_y = sc.inverse_transform(pred)

4/4 [==============================] - 1s 23ms/step


In [ ]:
plt.plot(train_y,color = 'red', label = 'Real Price')
plt.plot(test_y, color = 'blue', label = 'Predicted Price')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Google Stock Price')
plt.legend()
plt.show()